<a href="https://colab.research.google.com/github/james-monahan/Code-school-notebooks/blob/main/Week-13-bi-sql/SQL_and_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

SQL is based on a client-server architecture. 


Python can connect with every SQL databases (Oracle MySQL, Microsoft, etc...). SQLite is a little bit particular : it is a little file, who can play the server role, without username and password. It will be helpful for this quest. But keep in mind that you could connect your Python script with other databases, following the same steps.

In [ ]:
# Here we just import the SQLite file. This part is specific for SQLite.
import requests
r = requests.get('https://raw.githubusercontent.com/murpi/wilddata/master/quests/toys_and_models.sqlite')
open('toys_and_models.sqlite', 'wb').write(r.content)

307200

In [ ]:
!ls

sample_data  toys_and_models.sqlite


In [ ]:
# This part is common : we have to initiate a connection between Python and the SQL Database
import sqlite3

# This is the "connector". For secured database, it will be here that  you indicate your username and password.
conn = sqlite3.connect('toys_and_models.sqlite')

# This is the "cursor". A cursor is an object that can execute a query and retrieve the content of the result.
cursor = conn.cursor()


In [ ]:
# You can execute a query.
# The result is returned line by line. You can "fetch all" lines.
cursor.execute("select * from productlines").fetchall()

[('Classic Cars',
  'Attention car enthusiasts: Make your wildest car ownership dreams come true. Whether you are looking for classic muscle cars, dream sports cars or movie-inspired miniatures, you will find great choices in this category. These replicas feature superb attention to detail and craftsmanship and offer features such as working steering system, opening forward compartment, opening rear trunk with removable spare wheel, 4-wheel independent spring suspension, and so on. The models range in size from 1:10 to 1:24 scale and include numerous limited edition and several out-of-production vehicles. All models include a certificate of authenticity from their manufacturers and come fully assembled and ready for display in the home or office.',
  None,
  None),
 ('Motorcycles',
  'Our motorcycles are state of the art replicas of classic as well as contemporary motorcycle legends such as Harley Davidson, Ducati and Vespa. Models contain stunning details such as official logos, rotat

In [ ]:
# Pandas can import the result of a query
import pandas as pd
pd.DataFrame(cursor.execute("select * from productlines").fetchall())

,0,1,2,3
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [ ]:
# But it could be easier : you can use the pandas read_sql function.
pd.read_sql("select * from productlines", conn)

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [ ]:
# Here is the schema of the database
pd.read_sql("SELECT * FROM sqlite_master where type = 'table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,customers,customers,2,CREATE TABLE `customers` (\n `customerNumber`...
1,table,employees,employees,8,CREATE TABLE `employees` (\n `employeeNumber`...
2,table,offices,offices,10,CREATE TABLE `offices` (\n `officeCode` varch...
3,table,orderdetails,orderdetails,12,CREATE TABLE `orderdetails` (\n `orderNumber`...
4,table,orders,orders,49,CREATE TABLE `orders` (\n `orderNumber` int(1...
5,table,payments,payments,57,CREATE TABLE `payments` (\n `customerNumber` ...
6,table,productlines,productlines,64,CREATE TABLE `productlines` (\n `productLine`...
7,table,products,products,66,CREATE TABLE `products` (\n `productCode` var...


![Texte alternatif…](https://sp.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

# It's your turn

- First you have to create a DataFrame with the 5 most ordered productcodes
- Then you have to create a second DataFrame with stocks by products
- Finally, you will make a merge (with Pandas) between both DataFrame to display the stocks of the 5 most ordered products 

In [ ]:
top_5 = pd.read_sql("select productCode, SUM(quantityOrdered) AS TotalOrdered from orderdetails  \
            GROUP BY quantityOrdered \
            ORDER BY TotalOrdered DESC \
            LIMIT 5", conn)
top_5

,productCode,TotalOrdered
0,S32_3522,4464
1,S24_3371,4416
2,S18_4721,4214
3,S12_1099,3870
4,S24_3191,3828


In [ ]:
# Your code here :
in_stock = pd.read_sql("select productCode, SUM(quantityInStock) AS TotalStock from products \
            GROUP BY productCode", conn)
in_stock.head()

,productCode,TotalStock
0,S10_1678,7933
1,S10_1949,7305
2,S10_2016,6625
3,S10_4698,5582
4,S10_4757,3252


In [ ]:
five_by_inventory = top_5.merge(in_stock, left_on='productCode', right_on='productCode', how='inner')
five_by_inventory

,productCode,TotalOrdered,TotalStock
0,S32_3522,4464,814
1,S24_3371,4416,7995
2,S18_4721,4214,1249
3,S12_1099,3870,68
4,S24_3191,3828,4695


Good, but not optimized. Your "merge" with pandas is pretty the same as the "join" in SQL. Moreover, your script has to download the whole product stocks table. 

So, now, you have to write a SQL query to have directly the stocks of the 5 most ordered products. (So, you have to "join" both tables in the SQL Query)

In [ ]:
top_5_direct = pd.read_sql("select od.productCode, SUM(od.quantityOrdered) AS TotalOrdered, \
            pc.quantityInStock AS InStock\
            FROM orderdetails AS od  \
            JOIN products AS pc \
            ON pc.productCode = od.productCode \
            GROUP BY quantityOrdered \
            ORDER BY TotalOrdered DESC \
            LIMIT 5", conn)

top_5_direct

,productCode,TotalOrdered,InStock
0,S32_3522,4464,814
1,S24_3371,4416,7995
2,S18_4721,4214,1249
3,S12_1099,3870,68
4,S24_3191,3828,4695


Both methods have same results ? Congrats ! 

# Go forward

If you want, you can try to connect to the MySQL Server with your computer. To do this, you need to have MySQL Server on your computer, and to execute your python script from your computer too (so on Jupyterlab or on Spyder, **but not on google colab**).
Below, script to connect to your MySQL Server.
And here, [the SQL script to execute](https://drive.google.com/file/d/1_YtQlHCoGCQpLytXFpk6170WnCXb8hRz/view?usp=sharing) in the workbench if you don't have the database.

In [ ]:
# !pip install mysql-connector-python

# import mysql.connector
# import pandas as pd

# conn = mysql.connector.connect(user='YOUR_USERNAME', password='YOUR_PASSWORD',
#                               host='localhost',
#                               database='toys_and_models')

# pd.read_sql('select * from orders', conn)